In [ ]:
!pip install transformers
# !pip install datasets
# !pip install sentence-transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import nltk
import os
from tqdm import tqdm
from torch.autograd import Variable 
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import string
import re
import seaborn as sns
import pickle
# from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

     |████████████████████████████████| 4.0 MB 12.5 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 6.6 MB 39.1 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        
class CustomDataset(Dataset):
    def __init__(self, df,  tokenizer, transform=None, target_transform=None):
        self.features = df[['Definition1', 'Definition2']]
        self.labels = df['Binary labels']
        self.transform = transform
        self.target_transform = target_transform
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        def1 = self.features.loc[idx]['Definition1']
        def2 = self.features.loc[idx]['Definition2']
        inputs = self.tokenizer.encode_plus(
            def1, def2, 
            add_special_tokens=True,
            padding='max_length',
            max_length = 512,
            truncation=True   
        )
        return {
            "ids": torch.tensor(inputs["input_ids"], dtype=torch.long),
            "mask": torch.tensor(inputs["attention_mask"], dtype=torch.long),
            "targets": torch.tensor((self.labels.loc[idx]), dtype=torch.float)
        }


base_path = '/content/drive/MyDrive/DL Assignment 3/'

path_to_train = '/content/drive/MyDrive/DL Assignment 3/PARADE_dataset-main/PARADE_train.txt'
path_to_test = '/content/drive/MyDrive/DL Assignment 3/PARADE_dataset-main/PARADE_test.txt'
path_to_val ='/content/drive/MyDrive/DL Assignment 3/PARADE_dataset-main/PARADE_validation.txt'

df_train = pd.read_csv(path_to_train, delimiter='\t')
df_test = pd.read_csv(path_to_test, delimiter='\t')
df_val = pd.read_csv(path_to_val, delimiter='\t')

train_data = CustomDataset(df_train, tokenizer)
test_data = CustomDataset(df_test, tokenizer)
val_data = CustomDataset(df_val, tokenizer) 

batch_size = 64

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(dataset=val_data , batch_size=batch_size, shuffle=True, drop_last=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_def1 = df_train['Definition1']
train_def2 = df_train['Definition2']

val_def1 = df_val['Definition1']
val_def2 = df_val['Definition2']

test_def1 = df_test['Definition1']
test_def2 = df_test['Definition2']

print("Train statistics")
print("Average length of Definition1: ", np.mean(train_def1.str.len()))
print("Max length of Definition1 : ", max(train_def1.str.len()))
print("Min length of Definition1 : ", min(train_def1.str.len()))
print("Median length of Definition1 : ", np.median(train_def1.str.len()))

print("Average length of Definition2 : ", np.mean(train_def2.str.len()))
print("Max length of Definition2 : ", max(train_def2.str.len()))
print("Min length of Definition2 : ", min(train_def2.str.len()))
print("Median length of Definition2 : ", np.median(train_def2.str.len()))

print("Test statistics")
print("Average length of Definition1: ", np.mean(test_def1.str.len()))
print("Max length of Definition1 : ", max(test_def1.str.len()))
print("Min length of Definition1 : ", min(test_def1.str.len()))
print("Median length of Definition1 : ", np.median(test_def1.str.len()))

print("Average length of Definition2 : ", np.mean(test_def2.str.len()))
print("Max length of Definition2 : ", max(test_def2.str.len()))
print("Min length of Definition2 : ", min(test_def2.str.len()))
print("Median length of Definition2 : ", np.median(test_def2.str.len()))

print("Validation statistics")
print("Average length of Definition1: ", np.mean(val_def1.str.len()))
print("Max length of Definition1 : ", max(val_def1.str.len()))
print("Min length of Definition1 : ", min(val_def1.str.len()))
print("Median length of Definition1 : ", np.median(val_def1.str.len()))

print("Average length of Definition2 : ", np.mean(val_def2.str.len()))
print("Max length of Definition2 : ", max(val_def2.str.len()))
print("Min length of Definition2 : ", min(val_def2.str.len()))
print("Median length of Definition2 : ", np.median(val_def2.str.len()))

Train statistics
Average length of Definition1:  108.85509933774834
Max length of Definition1 :  259
Min length of Definition1 :  15
Median length of Definition1 :  106.5
Average length of Definition2 :  108.31615894039736
Max length of Definition2 :  253
Min length of Definition2 :  14
Median length of Definition2 :  104.0
Test statistics
Average length of Definition1:  110.34193072955048
Max length of Definition1 :  227
Min length of Definition1 :  17
Median length of Definition1 :  107.0
Average length of Definition2 :  108.8548268238762
Max length of Definition2 :  234
Min length of Definition2 :  17
Median length of Definition2 :  104.0
Validation statistics
Average length of Definition1:  108.68549019607843
Max length of Definition1 :  227
Min length of Definition1 :  23
Median length of Definition1 :  104.0
Average length of Definition2 :  105.60392156862746
Max length of Definition2 :  243
Min length of Definition2 :  15
Median length of Definition2 :  102.0


In [ ]:
def convertSigmoid(tens):
  tens[tens>0.5] = 1
  tens[tens<=0.5] = 0

  return tens

def trainModel(model, optimizer, device, criteria, train_loader, val_loader, test_loader, epochs, loss_plot_path = None, model_name = None, model_save_path = None):

    batch_size = 0
    loss_vs_epochs = []
    val_loss_vs_epochs = []
    train_acc_vs_epochs = []
    val_acc_vs_epochs = []

    for epoch in range(epochs):
        print("Epoch "+str(epoch+1))
        total_samples = 0
        correct = 0
        cur_epoch_loss = 0
        for batch_idx, batch in enumerate(tqdm(train_loader)):

            ids = batch["ids"].to(device, dtype=torch.long)
            mask = batch["mask"].to(device, dtype=torch.long)
            targets = batch["targets"].to(device, dtype=torch.float)
            total_samples += len(targets)

            print(mask.size())
            print(ids.size())
            # forward
            output = model(ids, mask)
            loss = criteria(output, targets)
            
            # backward
            optimizer.zero_grad()
            loss.backward()
            cur_epoch_loss += loss.cpu().detach().numpy()
            
            # gradient descent or adam step
            optimizer.step()
            
            required_tens = convertSigmoid(output)
            # Total correct predictions
            correct += (required_tens == targets).sum()
        
        correct = correct.cpu().detach().numpy()
        loss_vs_epochs.append(cur_epoch_loss/total_samples)
        
        train_acc = 100*correct/total_samples
        print("Train loss : " + str(loss_vs_epochs[-1]))
        print("Train Accuracy : " + str(train_acc))
        train_acc_vs_epochs.append(train_acc)
        val_loss, val_acc = testModel(model, val_loader, criteria, is_val = True)
        val_loss_vs_epochs.append(val_loss)
        val_acc_vs_epochs.append(val_acc)

        print()
        # testModel(distil_bert, model, val_loader, criteria, is_val = False)

    if(loss_plot_path):
        plt.plot(train_acc_vs_epochs, c='r')
        plt.plot(val_acc_vs_epochs, c='b')
        plt.legend(['Train accuracy', 'Validation accuracy'])
        if(model_name):
            plt.title("Model : "+model_name)
        
        plt.savefig(loss_plot_path + model_name + '_accuracy_plots.png')
        plt.show()
        plt.clf()

        plt.plot(loss_vs_epochs, c='r')
        plt.plot(val_loss_vs_epochs, c='b')
        plt.legend(['Train loss', 'Validation loss'])
        if(model_name):
            plt.title("Model : "+model_name)
        
        plt.savefig(loss_plot_path + model_name + '_training_loss.png')
        plt.show()
        plt.clf()

    if(model_save_path):
        torch.save(model.state_dict(), model_save_path + model_name + '.pkl')


def testModel(model, test_loader, criteria, is_val = False):
    correct = 0 
    cur_loss = 0
    total = 0

    full_predict = []
    full_outputs = []

    for batch_idx, batch in enumerate(tqdm(test_loader)):
        ids = batch["ids"].to(device, dtype=torch.long)
        mask = batch["mask"].to(device, dtype=torch.long)
        targets = batch["targets"].to(device, dtype=torch.float)

        output = model(ids, mask)
        loss = criteria(output, targets).item()
        cur_loss += loss

        output = output.to(device = 'cpu')
        targets = targets.to(device = 'cpu')
        
        full_predict.extend(output.cpu().detach().numpy().tolist())
        full_outputs.extend(targets.cpu().detach().numpy().tolist())

    full_predict = convertSigmoid(np.array(full_predict))
    total = len(full_predict)
    correct = (full_outputs == full_predict).sum()

    model.train()
    if(is_val):
        loss = cur_loss/total
        accuracy = 100*correct / total
        print("Validation loss : "+str(loss))
        print("Validation accuracy : " + str(accuracy))
        return loss, accuracy

    else:
        print()
        target_names = ['class 0', 'class 1' ]
        print(classification_report(full_outputs, full_predict, target_names=target_names))
        print("F1 score: ")
        print(f1_score(full_outputs, full_predict, average='macro'))
        loss = cur_loss/total
        accuracy = 100*correct / total
        print("Test loss : "+str(loss))
        print("Test accuracy : " + str(accuracy))

def load_saved_model(model_name : str, device : str, model: torch.nn, base_path : str):
    model.load_state_dict(torch.load(base_path + model_name, map_location=torch.device(device)))
    model.eval()
    return model

In [ ]:
class myModel(nn.Module):
    def __init__(self, sent_len = 512, unfreeze = 10):
        super(myModel, self).__init__()
        
        self.sent_len = sent_len
        self.gru = nn.GRU(input_size=768, hidden_size=768, num_layers=2, bidirectional=True, batch_first=True)
        
        self.distil_bert = DistilBertModel.from_pretrained("distilbert-base-uncased",).to(device=device)
        
        self.total_trainable_layers_distil_bert = []
        self.freeze_count = 0

        for name, param in self.distil_bert.named_parameters():
          if param.requires_grad:
              self.total_trainable_layers_distil_bert.append(name)

              if(self.freeze_count + unfreeze < 100):
                  param.requires_grad = False
                  self.freeze_count += 1

        print("Total trainable distil bert layers are : " + str(len(self.total_trainable_layers_distil_bert)))
        print("Layers freezed = "+str(self.freeze_count))
        
        self.l1 = nn.Linear(2*768, 128)
        self.l2 = nn.Linear(128, 1)

    def forward(self, ids, mask):
        embed = self.distil_bert(ids, attention_mask = mask)
        embed = embed[0]

        e_cat, _ = self.gru(embed)
        e_cat = e_cat.max(dim = 1)[0]
        e_cat = torch.squeeze(e_cat)

        e_cat = F.elu(self.l1(e_cat))
        e_cat = F.elu(self.l2(e_cat))
        e_cat = e_cat.view(-1)

        return torch.sigmoid(e_cat)

In [ ]:
model = myModel(sent_len = 512, unfreeze = 0).to(device=device)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total trainable distil bert layers are : 100
Layers freezed = 100


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

model_name = 'gru_max_across_dim1'
epochs = 5
trainModel(model, optimizer, device, criterion, train_loader, val_loader, test_loader, epochs, loss_plot_path = base_path, model_name = model_name, model_save_path = base_path)

testModel(model, test_loader, criterion, is_val = False)

100%|██████████| 21/21 [00:33<00:00,  1.60s/it]


              precision    recall  f1-score   support

     class 0       0.64      0.89      0.75       703
     class 1       0.80      0.46      0.58       641

    accuracy                           0.69      1344
   macro avg       0.72      0.68      0.67      1344
weighted avg       0.72      0.69      0.67      1344

F1 score: 
0.6662685415704531
Test loss : 0.009357376999798276
Test accuracy : 68.67559523809524
